In [1]:
print("""
@Description: 从张量开始
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-08-17 15:45:29
""")


@Description: 从张量开始
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-08-17 15:45:29



## 张量：多维数组

### 从 Python 列表到 PyTorch 张量

In [1]:
a = [1.0, 2.0, 1.0]
a[0]

1.0

In [3]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

### 构造第 1 个张量

In [4]:
import torch
# 创建一个大小为 3 的一维张量，用 1.0 填充
a = torch.ones(3)
a

In [5]:
a[1]

tensor(1.)

In [6]:
float(a[0])

1.0

In [7]:
a[2] = 2.0
a

tensor([1., 1., 2.])

### 张量的本质

In [8]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

In [9]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [10]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [11]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [12]:
points.shape

torch.Size([3, 2])

In [14]:
points = torch.zeros(3, 2)

In [15]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [16]:
points[0]

tensor([4., 1.])

## 索引张量

In [17]:
some_list = list(range(6))
some_list[:]

[0, 1, 2, 3, 4, 5]

In [18]:
some_list[1:4]
# 包含第 1 个元素到第 3 个元素，不包含第 4 个元素

[1, 2, 3]

In [19]:
some_list[1:]
# 包含第 1 个元素到列表末尾的元素

[1, 2, 3, 4, 5]

In [20]:
some_list[:4]
# 从列表开始到第 3 个元素，不包含第 4 个元素

[0, 1, 2, 3]

In [21]:
some_list[:-1]
# 从列表开始到第 3 个元素，不包含第 4 个元素

[0, 1, 2, 3, 4]

In [22]:
some_list[1:4:2]
# 从第 1 个元素（包含）到第 4 个元素（不包含），移动步长为 2

[1, 3]

In [23]:
points[1:]

tensor([[5., 3.],
        [2., 1.]])

In [24]:
points[1:, :]

tensor([[5., 3.],
        [2., 1.]])

In [25]:
points[1:, 0]

tensor([5., 2.])

In [26]:
points[None]
# 增加大小为 1 的维度，就像 unsqueeze()方法一样

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

## 命名张量

In [28]:
img_t = torch.randn(3, 5, 5) # shape [channels, row, columns]
weights = torch.tensor([0.2126, .7152, .0722])

In [29]:
batch_t = torch.randn(2, 3, 5, 5)

RGB 通道有时在第 0 维，有时在第一维，但我们可以通过从末端开始计数来归纳：它们总是在从末端开始的第三维中。因此，惰性的未加权平均值可以写成如下的形式：

In [31]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

PyTorch 将允许我们对相同形状的张量进行乘法运算，也允许与给定维度中其中一个操作数大小为 1 的张量进行运算。它还会自动附加大小为 1 的前导维度，这个特性被称为广播。形状为(2,3,5,5)的 batch_t 乘一个形状为(3,1,1) unsqueezed_weights 张量，得到一个形状为(2,3,5,5)的张量，由此我们可以从末端开始对第三维（3 个通道）进行求和：

In [46]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)

In [49]:
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

程序很快变得混乱

In [52]:
img_gray_weighted_fancy = torch.einsum('...chw, c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw, c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [65]:
print(torch.__version__)
# 2.0.1 仍存在试验性特征
weights_named = torch.tensor([.2126, .7251, .0722], names=['channels'])
weights_named

2.0.1+cpu


tensor([0.2126, 0.7251, 0.0722], names=('channels',))

In [61]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [67]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

接收维度参数的函数，例如 sum()，也接收命名维度：

In [70]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [75]:
try:
    (img_named[..., :3] * weights_named).sum("channels")
except RuntimeError:
    print("如果我们试图用不同的名称组合维度，会出现一个错误")

如果我们试图用不同的名称组合维度，会出现一个错误


In [76]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

## 张量的元素类型

### 使用 dtype 指定数字类型

### 适合任何场合的 dtype

### 管理张量的 dtype 属性

In [77]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [78]:
short_points.dtype

torch.int16

In [ ]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()